# Webscraping from Linkedin 

This is a surface level implementation that will only look for titles for the time being

In [2]:
import requests
import time
from bs4 import BeautifulSoup
import random

In [ ]:
url = 'https://www.linkedin.com/jobs/search'
param = {
    'keywords': '"Generative AI" OR "LLM" OR "Large Language Model" OR "Prompt Engineering" OR "Foundation Model" OR "Transformer" OR "RAG" OR "Reinforcement Learning With Human Feedback" OR "RLHF"',
    'location': 'United States',
    'f_TPR': 'r604800'  # past week
}
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Accept-Language': 'en-US'
}

start = 0
maxi = 2  # max empty pages allowed
empty = 0
titles = []
jobid = []

while empty < maxi:
    param['start'] = start

    print(f"\n Jobs num {start + 1} to {start + 25}...")

    r = requests.get(url, params=param, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    job_cards = soup.select('li div[data-entity-urn^="urn:li:jobPosting:"]')

    if not job_cards:
        empty += 1
        print("No results,", empty,"pages in a row")

    else:
        for card in job_cards:
            job_id = card['data-entity-urn'].split(':')[-1] # get job id for each row
            title_tag = card.select_one('span.sr-only') # get job title for each row
            title = title_tag.get_text(strip=True) if title_tag else "N/A" #safeguard against errors
            titles.append(title)
            jobid.extend(job_id)
        empty = 0  # reset if jobs found

    start += 25
    time.sleep(random.uniform(3, 6))  # random delay




 Jobs num 1 to 25...

 Jobs num 26 to 50...

 Jobs num 51 to 75...

 Jobs num 76 to 100...

 Jobs num 101 to 125...

 Jobs num 126 to 150...

 Jobs num 151 to 175...

 Jobs num 176 to 200...

 Jobs num 201 to 225...

 Jobs num 226 to 250...

 Jobs num 251 to 275...

 Jobs num 276 to 300...

 Jobs num 301 to 325...

 Jobs num 326 to 350...

 Jobs num 351 to 375...

 Jobs num 376 to 400...

 Jobs num 401 to 425...

 Jobs num 426 to 450...

 Jobs num 451 to 475...

 Jobs num 476 to 500...

 Jobs num 501 to 525...

 Jobs num 526 to 550...

 Jobs num 551 to 575...

 Jobs num 576 to 600...

 Jobs num 601 to 625...

 Jobs num 626 to 650...

 Jobs num 651 to 675...

 Jobs num 676 to 700...

 Jobs num 701 to 725...

 Jobs num 726 to 750...

 Jobs num 751 to 775...

 Jobs num 776 to 800...

 Jobs num 801 to 825...

 Jobs num 826 to 850...

 Jobs num 851 to 875...

 Jobs num 876 to 900...

 Jobs num 901 to 925...

 Jobs num 926 to 950...

 Jobs num 951 to 975...

 Jobs num 976 to 1000...

 Jobs 

In [ ]:
len(titles) #maybe duplicates?

19169

In [13]:
print(set(titles))

{'Machine Learning Engineer with Generative AI', 'Gen AI Lead', 'Applied Artificial Intelligence Machine Learning Lead, VP', 'AI Research Scientist - Post Training', 'NLP Research Engineer Intern', 'Research Scientist, Applied AI/ML, X', 'Machine Learning Engineer', 'Machine Learning Research Scientist', 'Gen AI Engineer - Marquee - Associate - NYC - GBM Public', 'Machine Learning Research Engineer', 'Machine Learning Engineers', 'Director of Innovation, AI', 'Gen AI ML Engineer', 'AI Machine Learning Engineer - Query Understanding', 'Senior AI Engineer, Agentic AI', 'Data Scientist, Remote', 'Machine Learning Engineer (Data)', 'Senior Research Scientist: Robot Manipulation Learning ...', '1.83 Robotics Software Engineer: LiDAR Mapping', 'AI  Software Engineer', 'Machine Learning Researchers (Reinforcement Learning) - Open Level', 'Artificial Intelligence Consultant', 'Generative AI Architect', 'ML Engineer', 'AI Engineer', 'ML Engineer (Gen AI/Computer Vision) - Quantitative Trading',